Go to Google Takeout and download just your emails. It'll likely take a couple of minutes to get the email with your downloaded data. Then download it and save the path.

# **Code to convert mbox files to csv**

In [ ]:
'''import mailbox
import pandas as pd
from email.utils import parsedate_to_datetime


# Open the mbox file
mbox = mailbox.mbox("Users/nmall/Desktop/All mail Including Spam and Trash-002 (1).mbox") # change to the path of your mbox file

data = []
for message in mbox:
    subject = message["subject"]
    sender = message["from"]
    recipient = message["to"]
    date = message["date"]

    # Convert date to datetime format
    date = parsedate_to_datetime(date) if date else None

    # Extract email body
    if message.is_multipart():
        body = ""
        for part in message.walk():
            if part.get_content_type() == "text/plain":  # Extract plain text only
                body += part.get_payload(decode=True).decode(errors="ignore")
    else:
        body = message.get_payload(decode=True).decode(errors="ignore")

    data.append([subject, sender, recipient, date, body])

# Convert to DataFrame
df = pd.DataFrame(data, columns=["Subject", "Sender", "Recipient", "Date", "Body"])
df.to_csv('emails.csv', index=False)
'''

'import mailbox\nimport pandas as pd\nfrom email.utils import parsedate_to_datetime\n\n\n# Open the mbox file\nmbox = mailbox.mbox("Users/nmall/Desktop/All mail Including Spam and Trash-002 (1).mbox") # change to the path of your mbox file\n\ndata = []\nfor message in mbox:\n    subject = message["subject"]\n    sender = message["from"]\n    recipient = message["to"]\n    date = message["date"]\n\n    # Convert date to datetime format\n    date = parsedate_to_datetime(date) if date else None\n\n    # Extract email body\n    if message.is_multipart():\n        body = ""\n        for part in message.walk():\n            if part.get_content_type() == "text/plain":  # Extract plain text only\n                body += part.get_payload(decode=True).decode(errors="ignore")\n    else:\n        body = message.get_payload(decode=True).decode(errors="ignore")\n\n    data.append([subject, sender, recipient, date, body])\n\n# Convert to DataFrame\ndf = pd.DataFrame(data, columns=["Subject", "Sender"

In [ ]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from html import unescape
from email.header import decode_header

# Function to clean a full Pandas column
def clean_column(column):
    return column.astype(str).apply(remove_unwanted_chars)

# Function to remove unwanted Unicode characters
def remove_unwanted_chars(text):
    if not isinstance(text, str):
        return ""

    # Remove Zero Width characters, Byte Order Marks (BOM), etc.
    text = re.sub(r'[\u200B-\u200F\uFEFF]', '', text)

    # Remove non-printable and control characters (keeping only printable ASCII)
    text = re.sub(r'[^\x20-\x7E]', '', text)

    return text.strip()

# Function to clean email body column
def clean_email_body_column(column):
    return column.astype(str).apply(clean_email_body)

# Function to clean email body text
def clean_email_body(html):
    if not isinstance(html, str):
        return ""

    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(separator=" ")
    text = unescape(text)
    text = re.sub(r'\s+', ' ', text).strip()

    return remove_unwanted_chars(text)

# Function to decode a Pandas column (for email subjects or sender names)
def decode_column(column):
    return column.astype(str).apply(decode)

# Function to decode text (for email subjects, names, etc.)
def decode(encoded_text):
    decoded_parts = decode_header(encoded_text)
    decoded_str = ""

    for part, encoding in decoded_parts:
        if isinstance(part, bytes):
            decoded_str += part.decode(encoding or "utf-8", errors="ignore")
        else:
            decoded_str += part
    return remove_unwanted_chars(decoded_str.strip())

# Function to clean and decode subject lines in a Pandas column
def decode_subject_column(column):
    return column.astype(str).apply(decode_subject)

# Function to decode and clean email subjects
def decode_subject(encoded_text):
    if not isinstance(encoded_text, str):
        return ""
    decoded_parts = decode_header(encoded_text)
    decoded_str = ""

    for part, encoding in decoded_parts:
        try:
            if isinstance(part, bytes):
                encoding = encoding or "utf-8"
                decoded_str += part.decode(encoding, errors="ignore")
            else:
                decoded_str += part
        except LookupError:
            decoded_str += str(part)

    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', decoded_str)
    return remove_unwanted_chars(cleaned_text.strip())

def extract_sender_name(sender):
    # Split by '<' and take the first part
    return sender.split('<')[0].strip()
def extract_date(date_str):
    # Split by space and take the first part (the date)
    return date_str.split(' ')[0]

# Read CSV safely
df = pd.read_csv("emails.csv", encoding="utf-8")
# Apply cleaning functions to "Subject", "Sender", and "Body"
df["Subject"] = decode_subject_column(df["Subject"])
df["Sender"] = decode_column(df["Sender"])
df["Body"] = clean_email_body_column(df["Body"])
df["Sender"] = df["Sender"].apply(extract_sender_name)
df["Date"] = df["Date"].apply(extract_date)


print(df.head())
# Save cleaned file


<ipython-input-3-93b5d0337e59>:33: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(html, "html.parser")


                                       Subject              Sender  \
0                     Whole Foods gets a union       LinkedIn News   
1        Find Textbook Answers by REAL Experts          "Numerade"   
2  Hear that sizzle Citrus Lime Shrimp is BACK  QDOBA Mexican Eats   
3                Hi Nick from LinkedIn Premium      Curtis Coatman   
4                 New Data Analyst Job Matches            Built In   

                           Recipient        Date  \
0  Nick Malloy <nmalloy13@gmail.com>  2025-01-28   
1                nmalloy13@gmail.com  2025-01-28   
2                nmalloy13@gmail.com  2025-01-29   
3                nmalloy13@gmail.com  2025-01-29   
4                nmalloy13@gmail.com  2025-01-29   

                                                Body  
0  ---------------------------------------- This ...  
1  Numerade's expert educators answer your most d...  
2  Hear that sizzle? Citrus Lime Shrimp is BACK. ...  
3  LinkedIn Your personal LinkedIn Customer Su

In [ ]:
# Drop rows where 'Body' is NaN
df = df[~df["Body"].str.contains("nan", na=False)]

# Remove rows where 'Body' contains the unwanted message
df = df[~df["Body"].str.contains("It looks like your email client might not support HTML formatted email", na=False)]
df.head()

,Subject,Sender,Date,Body
0,Whole Foods gets a union,LinkedIn News,2025-01-28,---------------------------------------- This ...
1,Find Textbook Answers by REAL Experts,"""Numerade""",2025-01-28,Numerade's expert educators answer your most d...
2,Hear that sizzle Citrus Lime Shrimp is BACK,QDOBA Mexican Eats,2025-01-29,Hear that sizzle? Citrus Lime Shrimp is BACK. ...
3,Hi Nick from LinkedIn Premium,Curtis Coatman,2025-01-29,LinkedIn Your personal LinkedIn Customer Succe...
5,Youd be a great fit for this Data Scientist ro...,Glassdoor Jobs,2025-01-27,Apply Now! ...


In [ ]:
# df = df.drop(columns = ["Recipient"])
df.to_csv("cleaned_emails.csv", encoding="utf-8", errors="replace", index=False)